In [1]:
import datetime
import json

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from rexsettings.settings import postgres_sequoia_db_uri, postgres_third_party_db_uri
import sqlalchemy as sa
import sqlalchemy.orm as orm

In [2]:
rex_market_area_name = 'Boston'
data_dir = 'index-team-data'

In [3]:
market_area_name = rex_market_area_name.lower().replace(' ', '_').replace('-', '_')
data_path = f'../{data_dir}/{market_area_name}'

In [4]:
sequoia_engine = sa.create_engine(postgres_sequoia_db_uri())
third_party_engine = sa.create_engine(postgres_third_party_db_uri())
sequoia_meta = sa.MetaData(bind=sequoia_engine)
third_party_meta = sa.MetaData(bind=third_party_engine)

In [5]:
sequoia_session = orm.sessionmaker(bind=sequoia_engine)()
third_party_session = orm.sessionmaker(bind=third_party_engine)()

In [6]:
table_name_to_info = {'listing': (sequoia_meta, 'rexdb'),
                      'market_area_lookup': (sequoia_meta, 'location'),
                      'property': (third_party_meta, 'mls')}

In [7]:
def get_table(table_name):
    info = table_name_to_info[table_name]
    return sa.Table(table_name, info[0], schema=info[1], autoload=True)

In [8]:
lstg = get_table('listing')
market_area_lookup = get_table('market_area_lookup')
mls_property = get_table('property')
mls_listing = sa.Table('listing', third_party_meta, schema='mls', autoload=True)

/Users/wfried/anaconda3/envs/digital_index/lib/python3.8/site-packages/sqlalchemy/dialects/postgresql/base.py:3185: SAWarning: Did not recognize type 'geometry' of column 'geo_point'
  util.warn(
/Users/wfried/anaconda3/envs/digital_index/lib/python3.8/site-packages/sqlalchemy/dialects/postgresql/base.py:3515: SAWarning: Skipped unsupported reflection of expression-based index property__geo_point__idx
  util.warn(


In [9]:
# map zipcode to REX market area
q = (sequoia_session
    .query(market_area_lookup.c.zipcode,
           market_area_lookup.c.market_area)
    .distinct()
    )

zipcode_to_market_area = {zipcode: market_area for zipcode, market_area in q}

In [10]:
market_area_zipcodes = {zipcode for zipcode, market_area in zipcode_to_market_area.items()
                        if market_area == rex_market_area_name}

In [11]:
with open(f'{data_path}/zipcodes.txt', 'w') as f:
    f.write(json.dumps(list(market_area_zipcodes)))

In [12]:
# limit query to months that have been fully observed
date_today = datetime.date.today()
latest_query_date = date_today.replace(day=1)

In [13]:
q = (third_party_session
     .query(mls_listing.c.cc_property_id,
            mls_property.c.zipcode,
            mls_property.c.latitude,
            mls_property.c.longitude,
            mls_listing.c.original_list_date,
            mls_listing.c.sale_date,
            mls_listing.c.withdrawn_date,
            mls_listing.c.expired_date,
            mls_listing.c.cancelled_date,
            mls_listing.c.current_status,
            mls_listing.c.original_list_price,
            mls_listing.c.sale_price)
     .join(mls_property, mls_property.c.cc_property_id == mls_listing.c.cc_property_id)
     .filter(mls_property.c.zipcode.in_(market_area_zipcodes))
     .filter(mls_listing.c.original_list_date < latest_query_date)
     .filter(sa.or_(mls_listing.c.sale_date.is_(None), mls_listing.c.sale_date < latest_query_date))
     .order_by(mls_listing.c.original_list_date)
    )

lst = [(prop_id, zipcode, lat, long, list_date, sale_date, withdrawn_date, expired_date, cancelled_date, 
      current_status, list_price, sale_price)
     for prop_id, zipcode, lat, long, list_date, sale_date, withdrawn_date, expired_date, cancelled_date,
            current_status, list_price, sale_price in q]

In [14]:
columns = ['property_id', 'zipcode', 'latitude', 'longitude', 'list_date', 'sale_date', 'withdrawn_date', 
           'expired_date', 'cancelled_date', 'current_status', 'list_price', 'sale_price']

listing_dates_df = pd.DataFrame(lst, columns=columns)

In [15]:
listing_dates_df.to_csv(f'{data_path}/listing_dates.csv', index=False)